# ETHZ: 227-0966-00L
# Quantitative Big Imaging
# April 12, 2018

## Advanced Shape and Texture

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["figure.dpi"] = 150
plt.rcParams["font.size"] = 14
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.style.use('ggplot')
sns.set_style("whitegrid", {'axes.grid': False})

Literature / Useful References
========================================================
### Books
- Jean Claude, Morphometry with R
- [Online](http://link.springer.com/book/10.1007%2F978-0-387-77789-4) through ETHZ
- [Buy it](http://www.amazon.com/Morphometrics-R-Use-Julien-Claude/dp/038777789X)
- John C. Russ, “The Image Processing Handbook”,(Boca Raton, CRC Press)
- Available [online](http://dx.doi.org/10.1201/9780203881095) within domain ethz.ch (or proxy.ethz.ch / public VPN) 

*** 
### Papers / Sites
- Thickness
 - [1] Hildebrand, T., & Ruegsegger, P. (1997). A new method for the model-independent assessment of thickness in three-dimensional images. Journal of Microscopy, 185(1), 67–75. doi:10.1046/j.1365-2818.1997.1340694.x
- Curvature
 - http://mathworld.wolfram.com/MeanCurvature.html
 - [2] "Computation of Surface Curvature from Range Images Using Geometrically Intrinsic Weights"*, T. Kurita and P. Boulanger, 1992.
 - [radiomics.io]

# Interfaces / Surfaces

Many physical and chemical processes occur at surfaces and interfaces and consequently these structures are important in material science and biology. For this lecture surface and interface will be used interchangebly and refers to the boundary between two different materials (calcified bone and soft tissue, magma and water, liquid and gas) Through segmentation we have identified the unique phases in the sample under investigation. 

- Segmentation identifying volumes (3D) or areas (2D)
- Interfaces are one dimension lower corresponding to surface area (3D) or perimeter (2D)
- Interfaces are important for 
 - connectivity of cell networks, particularly neurons
 - material science processes like coarsening or rheological behavior
 - chemical processes (surface-bound diffusion, catalyst action)

# Surface Area / Perimeter

We see that the dilation and erosion affects are strongly related to the surface area of an object: the more surface area the larger the affect of a single dilation or erosion step. 

# Meshing


Constructing a mesh for an image provides very different information than the image data itself. Most crucially this comes when looking at physical processes like deformation.

While the images are helpful for visualizing we rarely have models for quantifying how difficult it is to turn a pixel __off__

If the image is turned into a mesh we now have a list of vertices and edges. For these vertices and edges we can define forces. For example when looking at stress-strain relationships in mechanics using Hooke's Model 
$$ \vec{F}=k (\vec{x}_0-\vec{x}) $$ 
the force needed to stretch one of these edges is proportional to how far it is stretched. 

# Marching Cubes

### Why
Voxels are very poor approximations for the surface and are very rough (they are either normal to the x, y, or z axis and nothing between). Because of their inherently orthogonal surface normals, any analysis which utilizes the surface normal to calculate another value (growth, curvature, etc) is going to be very inaccurate at best and very wrong at worst.

### [How](http://en.wikipedia.org/wiki/Marching_cubes)
The image is processed one voxel at a time and the neighborhood (not quite the same is the morphological definition) is checked at every voxel. From this configuration of values, faces are added to the mesh to incorporate the most simple surface which would explain the values. 

[Marching tetrahedra](http://en.wikipedia.org/wiki/Marching_tetrahedra) is for some applications a better suited approach

Outline
========================================================

- Motivation (Why and How?)
- What are Distance Maps?
- Skeletons
 - Tortuosity
- What are thickness maps?
 - Thickness with Skeletons

***

- Watershed Segmentation
 - Connected Objects
- Curvature
 - Characteristic Shapes



Learning Objectives
========================================================

### Motivation (Why and How?)

- How do we measure distances between many objects?
- How can we extract topology of a structure?

- How can we measure sizes in complicated objects?
- How do we measure sizes relavant for diffusion or other local processes?

- How do we identify seperate objects when they are connected?
- How do we investigate surfaces in more detail and their shape?
- How can we compare shape of complex objects when they grow?
 - Are there characteristic shape metrics?

What did we want in the first place
========================================================
To simplify our data, but an ellipse model is __too__ simple for many shapes

So while bounding box and ellipse-based models are useful for many object and cells, they do a very poor job with the sample below.

```{r, fig.cap="Single Cell",fig.height=6}

cell.im<-jpeg::readJPEG("ext-figures/Cell_Colony.jpg")
cell.lab.df<-im.to.df(bwlabel(cell.im<.6))
size.histogram<-ddply(subset(cell.lab.df,val>0),.(val),function(c.label) data.frame(count=nrow(c.label)))
keep.vals<-subset(size.histogram,count>25)


cur.cell.df<-subset(cell.lab.df,val==155)
cell.pca<-pca.fun(cur.cell.df)
cell.ellipse<-vec.to.ellipse(cell.pca)
ggplot(cur.cell.df,aes(x=x,y=y))+
  geom_tile(color="black",fill="grey")+
  geom_path(data=bbox.fun(cur.cell.df),aes(x=xi,y=yi,color="Bounding\nBox"))+
  geom_path(data=cell.ellipse,aes(color="Ellipse"))+
  labs(title="Single Cell",color="Shape\nAnalysis\nMethod")+
  theme_bw(20)+coord_equal()+guides(fill=F)
```

***
### Why
- We assume an entity consists of connected pixels (wrong)
- We assume the objects are well modeled by an ellipse (also wrong)

### What to do?

- Is it 3 connected objects which should all be analzed seperately?
- If we could __divide it__, we could then analyze each spart as an ellipse
- Is it one network of objects and we want to know about the constrictions?
- Is it a cell or organelle with docking sites for cell?
- Neither extents nor anisotropy are very meaningful, we need a __more specific metric__

Distance Maps: What are they
===
```{r dist_map_code}
# Distance map code
# Fill Image code
# ... is for extra columns in the data set
fill.img.fn<-function(in.img,step.size=1,...) {
  xr<-range(in.img$x)
  yr<-range(in.img$y)
  ddply(expand.grid(x=seq(xr[1],xr[2],step.size),
                  y=seq(yr[1],yr[2],step.size)),
        .(x,y),
      function(c.pos) {
        ix<-c.pos$x[1]
        iy<-c.pos$y[1]
        nset<-subset(in.img,x==ix & y==iy)
        if(nrow(nset)<1) nset<-data.frame(x=ix,y=iy,val=0,...)
        nset
        })
}
fill.img<-fill.img.fn(cur.cell.df)
euclid.dist<-function(x,x0,y,y0) sqrt((x-x0)^2+(y-y0)^2)
man.dist<-function(x,x0,y,y0) abs(x-x0)+abs(y-y0)
dist.map<-function(in.df,fg.ph=155,bg.ph=0,dist.fn=euclid.dist) {
  foreground.df<-subset(in.df,val==fg.ph)
  background.df<-subset(in.df,val==bg.ph)
  ddply(foreground.df,.(x,y),function(c.pos) {
    ix<-c.pos$x[1]
    iy<-c.pos$y[1]
    # calculate the minimum distance to a background voxel
    data.frame(dist=min(
      with(background.df,dist.fn(x,ix,y,iy))
      ))
  })
}


# grid used for all datasets
x.grid<-seq(-10,10,length.out=81)
# function to make a test circle image
make.sph.img<-function(sph.list,in.grid=x.grid) {
  
  test.sph.img<-expand.grid(x=in.grid,y=in.grid)
  test.sph.img$val<-0
  
  for(i in 1:nrow(sph.list)) {
    test.sph.img$val<-with(sph.list[i,], # use the namespace for the current cirlce
                           with(test.sph.img, # use the grid namespace as well
                                ifelse(val,1,ifelse(((x-xi)^2+(y-yi)^2)<ri^2,1,0))
                                ))
    }
  test.sph.img
  }

```

A map (or image) of distances. Each point in the map is the distance that point is from a given feature of interest (surface of an object, ROI, center of object, etc)

```{r, fig.cap="Simple Circles",fig.height=5}
test.sph.list<-data.frame(xi=c(-3,-3, 5, 5),
                    yi=c(-5, 5,-3, 5),
                    ri=c( 2, 3, 5, 2))
test.img<-make.sph.img(test.sph.list,seq(-2,8))

ggplot(test.img,aes(x=x,y=y,fill=ifelse(val,"FG","BG")))+
  geom_tile(color="black")+
  geom_text(aes(label=ifelse(val,"FG","BG")))+
  labs(title="Object Distance Map",fill="Phase")+
  theme_bw(20)+coord_equal()
```

***
### Definition
If we start with an image as a collection of points divided into two categories 
- $Im(x,y)=$ {Foreground, Background}
- We can define a distance map operator ($dist$) that transforms the image into a distance map

$$ dist(\vec{x}) = \textrm{min}(||\vec{x}-\vec{y}|| \forall \vec{y} \in \textrm{Background}) $$

We will use Euclidean distance $||\vec{x}-\vec{y}||$ for this class but there are other metrics which make sense when dealing with other types of data like Manhattan/City-block or weighted metrics.

Distance Maps: Types
===
Using this rule a distance map can be made for the euclidean metric 
```{r, fig.cap="Simple Circles",fig.height=5}
ggplot(fill.img.fn(dist.map(test.img,fg.ph=1,bg.ph=0),dist=0),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  geom_text(aes(label=ifelse(dist>0,round(dist*10)/10,"Obj")))+
  th_fillmap.fn(4)+
  labs(title="Euclidean Distance",fill="Distance\nFrom\nObject")+
  theme_bw(20)+coord_equal()
```

***

Similarly the Manhattan or city block distance metric can be used where the distance is defined as 
$$ \sum_{i} |\vec{x}-\vec{y}|_i $$

```{r, fig.cap="Simple Circles",fig.height=5}
ggplot(fill.img.fn(dist.map(test.img,fg.ph=1,bg.ph=0,dist.fn=man.dist),dist=0),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  geom_text(aes(label=ifelse(dist>0,round(dist*10)/10,"Obj")))+
  th_fillmap.fn(4)+
  labs(title="Manhattan / City Block Distance",fill="Distance\nFrom\nObject")+
  theme_bw(20)+coord_equal()
```

Distance Maps: Step by Step
===

<center>
![Growing](ext-figures/growing.gif)
</center>

Distance Maps: Precaution
===

The distance map is one of the crictical points where the resolution of the imaging system is important. 
- We measure distances computationally in pixels or voxels
- but for them to have a meaning physically they must be converted
 - Isotropic imaging (1 $\mu$ m x 1$\mu$ m x 1 $\mu$ m) is __fine__
 
***

### Anisotropic
Ideally 

- as part of filtering, resample and convert to an isotropic scale. 

If that is not possible 

- custom distance map algorithms which use the side-lengths of the voxels to calculate distance rather than assuming 1x1x1


Distance Maps
===
```{r, fig.cap="Simple Circles",fig.height=4}
test.sph.list<-data.frame(xi=c(-3,-3, 5, 5),
                    yi=c(-5, 5,-3, 5),
                    ri=c( 2, 3, 5, 2))
test.img<-make.sph.img(test.sph.list)
ggplot(cbind(test.img,label=ifelse(test.img$val,"FG","BG")),aes(x=x,y=y,fill=label))+
  geom_tile(color="black")+
  labs(title="Foreground and Background",fill="Label")+
  theme_bw(20)+coord_equal()
```

We can create 2 distance maps

1. Foreground $\rightarrow$ Background 
 - Information about the objects size and interior

1. Background $\rightarrow$ Foreground 
 - Information about the distance / space between objects

***

```{r, fig.cap="Simple Circles",fig.height=5}
ggplot(dist.map(test.img,fg.ph=1,bg.ph=0),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Object Distance Map",fill="Distance\nFrom\nSurface")+
  th_fillmap.fn(6)+
  theme_bw(20)+coord_equal()
```

```{r, fig.cap="Simple Circles",fig.height=5}
ggplot(dist.map(test.img,fg.ph=0,bg.ph=1),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Background Distance Map",fill="Distance\nFrom\nSurface")+
  th_fillmap.fn(6)+
  theme_bw(20)+coord_equal()
```

Distance Map
===

One of the most useful components of the distance map is that it is _relatively_ insensitive to small changes in connectivity.
- Component Labeling would find radically different results for these two images
 - One has 4 small circles
 - One has 1 big blob
 
```{r, fig.cap="Simple Circles",fig.height=5}
test.sph.list<-data.frame(xi=c(-4,-3, 5, 5),
                    yi=c(-4, 5,-3, 5),
                    ri=c( 4, 5, 5, 3)+0)
ggplot(dist.map(make.sph.img(test.sph.list),fg.ph=1,bg.ph=0),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Circle Distance Map",fill="Distance\nFrom\nSurface")+
  theme_bw(20)+coord_equal()
```

***

```{r, fig.cap="Closer Circles",fig.height=4}
test.sph.list.big<-test.sph.list
# bring everything closer

test.sph.list.big$xi<-0.95*test.sph.list.big$xi
test.sph.list.big$yi<-0.95*test.sph.list.big$yi
ggplot(dist.map(make.sph.img(test.sph.list.big),fg.ph=1,bg.ph=0),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Same Circles, a bit closer",fill="Distance\nFrom\nSurface")+
  theme_bw(20)+coord_equal()
```

```{r, fig.cap="Circles Histogram",fig.height=5}

ggplot(rbind(
  cbind(dist.map(make.sph.img(test.sph.list),fg.ph=1,bg.ph=0),type="Circles"),
  cbind(dist.map(make.sph.img(test.sph.list.big),fg.ph=1,bg.ph=0),type="Closer Circles")),
  aes(x=dist,color=type))+
  geom_density(aes(y=..count..))+
  labs(x="Distance from Surface",y="Voxel Count",color="Image")+
  theme_bw(20)
```
Distance Map of Cell
===

### Foreground

```{r, fig.cap="Cell Distance Map",fig.height=6}
ggplot(dist.map(fill.img,fg.ph=155,bg.ph=0),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Single Cell",fill="Distance\nFrom\nSurface")+
  theme_bw(20)+coord_equal()
```

***

### Background

```{r, fig.cap="Cell Distance Map",fig.height=6}
ggplot(dist.map(fill.img,fg.ph=0,bg.ph=155),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Single Cell",fill="Distance\nFrom\nSurface")+
  theme_bw(20)+coord_equal()
```

Skeletonization / Networks
===

For some structures like cellular materials and trabecular bone, we want a more detailed analysis than just thickness. We want to know

- which structures are connected
- how they are connected
- express the network in a simple manner
 - quantify tortuosity
 - branching

***

```{r, fig.cap="Mesh Mask",fig.height=9}
test.mesh<-expand.grid(xi=seq(-10,10,length.out=5),yi=seq(-10,10,length.out=4))
test.mesh$ri<-runif(nrow(test.mesh),min=0.5,max=2)
test.mesh.im<-make.sph.img(test.mesh)
ggplot(subset(test.mesh.im,val==0),aes(x=x,y=y))+
  geom_tile(color="black",fill="grey")+
  labs(title="Object Mask")+
  theme_bw(20)+coord_equal()
```

Skeletonization
===

The first step is to take the distance transform the structure 
$$I_d(x,y) = \textrm{dist}(I(x,y))$$
We can see in this image there are already local maxima that form a sort of backbone which closely maps to what we are interested in.


```{r, fig.cap="Distance Map",fig.height=5}
test.mesh.dist<-dist.map(test.mesh.im,fg.ph=0,bg.ph=1)
ggplot(test.mesh.dist,aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Circle Distance Map",fill="Distance\nFrom\nSurface")+
  theme_bw(20)+coord_equal()
```

Skeletonization: Ridges
===

By using the laplacian filter as an approximate for the derivative operator which finds the values which high local gradients.

$$ \nabla I_{d}(x,y) = (\frac{\delta^2}{\delta x^2}+\frac{\delta^2}{\delta y^2})I_d \approx \underbrace{\begin{bmatrix}
-1 & -1 & -1 \\
-1 & 8 & -1 \\
-1 & -1 & -1
\end{bmatrix}}_{\textrm{Laplacian Kernel}} \otimes I_d(x,y) $$

```{r, fig.cap="Skeleton",fig.height=4.5}
# use a laplacian kernel
lap.kernel<-data.frame(xi=c(-1, 0, 1,-1, 0, 1,-1, 0, 1),
                       yi=c(-1,-1,-1, 0, 0, 0, 1, 1, 1),
                       wi=c(-1,-1,-1,-1, 8,-1,-1,-1,-1))
test.mesh.dif<-data.frame(x=c(),y=c(),val=c())
for(i in 1:nrow(lap.kernel)) {
  temp.df<-test.mesh.dist[,c("x","y")]
  temp.df$val<-with(lap.kernel[i,],test.mesh.dist$dist*wi)
  temp.df$x<-with(lap.kernel[i,],temp.df$x+diff(x.grid)[1]*xi)
  temp.df$y<-with(lap.kernel[i,],temp.df$y+diff(x.grid)[1]*yi)
  test.mesh.dif<-rbind(test.mesh.dif,temp.df)
}
test.mesh.dif<-ddply(test.mesh.dif,.(x,y),function(all.vals) {
  data.frame(val=sum(all.vals$val))
})
ggplot(test.mesh.dif,aes(x=x,y=y,fill=val))+
  geom_tile(color="black")+
  labs(title="Laplacian Image",fill="Laplacian/\nGradient")+
  theme_bw(20)+coord_equal()+scale_fill_gradient2()
```

Creating the skeleton
===

We can locate the local maxima of the structure by setting a minimum surface distance
$$I_d(x,y)>MIN_{DIST}$$
and combining it with a minimum slope value 
$$\nabla I_d(x,y) > MIN_{SLOPE}$$

***

### Thresholds
Harking back to our earlier lectures, this can be seen as a threshold on a feature vector representation of the entire dataset.
- We first make the dataset into a tuple

$$ \textrm{cImg}(x,y) = \langle \underbrace{I_d(x,y)}_1, \underbrace{\nabla I_d(x,y)}_2 \rangle $$

$$ \textrm{skelImage}(x,y) = $$
$$ \begin{cases}
1, & \textrm{cImg}_1(x,y)\geq MIN-DIST \\ 
 &  \& \textrm{ cImg}_2(x,y)\geq MIN-SLOPE \\
0, & \textrm{otherwise}
\end{cases}$$ 


Skeleton: Different Thresholds
===

```{r, fig.cap="Skeleton",fig.height=9}
merge.mesh.dif<-rbind(
  cbind(subset(test.mesh.dist,dist>2),type="Non-surface\nPoints",val=0),
  cbind(subset(test.mesh.dif,val>1),type="Slope\nPoints",dist=0)
  )

ggplot(merge.mesh.dif,aes(x=x,y=y,fill=type))+
  geom_tile(color="black",alpha=0.75)+
  labs(title="Min Slope=1, Min Dist=2",fill="Source")+
  theme_bw(20)+coord_equal()
```

***

```{r, fig.cap="Skeleton",fig.height=9}
merge.mesh.dif<-rbind(
  cbind(subset(test.mesh.dist,dist>1),type="Non-surface\nPoints",val=0),
  cbind(subset(test.mesh.dif,val>0),type="Slope\nPoints",dist=0)
  )
ggplot(merge.mesh.dif,aes(x=x,y=y,fill=type))+
  geom_tile(color="black",alpha=0.5)+
  labs(title="Min Slope=0, Min Dist=1",fill="Source")+
  theme_bw(20)+coord_equal()
```


Skeleton: Pruning 
===

```{r, fig.cap="Skeleton",fig.height=9}

overlap.im<-ddply(merge.mesh.dif,.(x,y),function(c.pts) {
  data.frame(cnt=nrow(c.pts), # points from how many images are present
             dist=max(c.pts$dist))
})

ggplot(subset(overlap.im,cnt==2),aes(x=x,y=y))+
  geom_tile(color="black",alpha=0.9,fill="red")+
  labs(title="Min Slope=0, Min Dist=1.5",fill="Source")+
  theme_bw(20)+coord_equal()
```

***

The structure is a overgrown

1. Stricter 'thresholds' 
 - Sensitive to 'threshold' selection
 - Resolution dependent
1. Dedicated pruning algorithms
 - Ideally model-based
 - Minimum branch length
 - Minimum branch width


Skeleton: Junction Overview 
===

With the skeleton which is ideally one voxel thick, we can characterize the junctions in the system by looking at the neighborhood of each voxel.

```{r simple_grid, fig.cap="Skeleton",fig.height=9}
simple_grid<-subset(
  expand.grid(x=c(0:10),y=c(0:10)),
  ((x-1) %% 5==0) | ((y-1) %% 5==0)
  )

make.junc<-function(in.skel,step.size=1,max.dist=1) {
  t.neighbors<-subset(
    expand.grid(xv=step.size*c(-1:1),yv=step.size*c(-1:1)),
    sqrt(xv^2+yv^2)<=(max.dist*step.size))
  spread.skel<-ddply(t.neighbors,.(xv,yv),function(c.off) {
    data.frame(x=in.skel$x+c.off[1,"xv"],
              y=in.skel$y+c.off[1,"yv"],
              has_orig=((c.off[1,"xv"]==0) & (c.off[1,"yv"]==0)))
  })
  f.skel<-subset(
    ddply(spread.skel,.(x,y),function(in.pt) data.frame(cnt=nrow(in.pt),
                                                        has_orig=max(in.pt$has_orig))),
    has_orig>0)
  f.skel$label<-with(f.skel,ifelse(cnt==2,"Terminal",ifelse(cnt==3,"Segment",ifelse(cnt>3,"Junction",""))))
  f.skel
}

pruned.skeleton<-simple_grid

junc.skel1<-make.junc(pruned.skeleton,step.size=1,max.dist=1)

ggplot(junc.skel1,aes(x=x,y=y,fill=label))+
  geom_tile(color="black",alpha=0.9)+
  labs(title="Cross Neighborhood",fill="Type")+
  theme_bw(20)+coord_equal()
```

***

As with _nearly_ every operation, the neighborhood we define is important and we can see that if we use a box neighborhood vs a cross neighborhood (4 vs 8 adjacent) we count diagonal connections differently

```{r applied_skel, fig.cap="Skeleton",fig.height=9}

junc.skel2<-make.junc(pruned.skeleton,step.size=1,max.dist=2)

ggplot(junc.skel2,aes(x=x,y=y,fill=label))+
  geom_tile(color="black",alpha=0.9)+
  labs(title="Box Neighborhood",fill="Type")+
  theme_bw(20)+coord_equal()
```

Skeleton: Junctions Applied
===

With the skeleton which is ideally one voxel thick, we can characterize the junctions in the system by looking at the neighborhood of each voxel.

```{r, fig.cap="Skeleton",fig.height=9}

pruned.skeleton<-subset(overlap.im,cnt==2)

junc.skel1<-make.junc(pruned.skeleton,step.size=0.25,max.dist=1)

ggplot(junc.skel1,aes(x=x,y=y,fill=label))+
  geom_tile(color="black",alpha=0.9)+
  labs(title="Cross Neighborhood",fill="Type")+
  theme_bw(20)+coord_equal()
```

***

As with _nearly_ every operation, the neighborhood we define is important and we can see that if we use a box neighborhood vs a cross neighborhood (4 vs 8 adjacent) we count diagonal connections differently

```{r, fig.cap="Skeleton",fig.height=9}

junc.skel2<-make.junc(pruned.skeleton,step.size=0.25,max.dist=2)

ggplot(junc.skel2,aes(x=x,y=y,fill=label))+
  geom_tile(color="black",alpha=0.9)+
  labs(title="Box Neighborhood",fill="Type")+
  theme_bw(20)+coord_equal()
```

Skeleton: Network Assessment
===

Once we have classified the skeleton into _terminals_, _segments_, and _junctions_ we can analyze the various components and assess metrics like connectivity, branching, and many other. 

The easist way is to further process the image by
- component labeling the different segments and joints
- determining the average length of each segment
- number of incoming segments at a given joint


***

Most of the other metrics can be simply counted

### Cross Neighborhood

```{r, results='asis'}

kable(aggregate(x~label,data=junc.skel1,FUN=length))
```

### Box Neighborhood

```{r, results='asis'}

kable(aggregate(x~label,data=junc.skel2,FUN=length))
```

Skeleton: Tortuosity
===

One of the more interesting ones in material science is called tortuosity and it is defined as the ratio between the arc-length of a _segment_ and the distance between its starting and ending points. 
$$ \tau = \frac{L}{C} $$

A high degree of [tortuosity](http://en.wikipedia.org/wiki/Tortuosity) indicates that the network is convoluted and is important when estimating or predicting flow rates.
Specifically
- in geology it is an indication that diffusion and fluid transport will occur more slowly
- in analytical chemistry it is utilized to perform size exclusion chromatography
- in vascular tissue it can be a sign of pathology.



Thickness Map: What is it?
===

Thickness is a metric for assessing the size and structure of objects in a very generic manner. 
For every point $\vec{x}$ in the image you find the largest sphere which:
- contains that point
- is entirely contained within the object

***

![Thickness Diagram](ext-figures/ThickOverview.png) 
Taken from [1]

- The image shows a typical object
- The sphere centered at point $p$ with a radius $r$ is the largest that fits

### Application

- Ideal for spherical and cylindrical objects since it shows their radius
- Also relevant for flow and diffusion since it can highlight bottlenecks in structure



Thickness Map
===
```{r thick_map}
thick.map<-function(dist.df,min.dist=0.25) {
  obj.vox<-subset(dist.df,dist>=0)
  full.thick.map<-ddply(subset(dist.df,dist>=min.dist),.(x,y),function(c.pos) {
    ix<-c.pos$x[1]
    iy<-c.pos$y[1]
    ir<-c.pos$dist[1]
    # spread the distance out to all points within the sphere
    # save the images inside the sphere
    cbind(
      subset(obj.vox,((x-ix)^2+(y-iy)^2)<ir^2)[,c("x","y")],
      sph.rad=ir
    )
  })
  ddply(full.thick.map,.(x,y),function(c.pos) {
    data.frame(x=c.pos$x[1],y=c.pos$y[1],sph.rad=max(c.pos$sph.rad))
  })
}
```

```{r, fig.cap="Cell Distance Map",fig.height=6}

dmap.fg<-dist.map(fill.img,fg.ph=155,bg.ph=0)

ggplot(dmap.fg,aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Single Cell",fill="Distance\nFrom\nSurface")+
  th_fillmap.fn(max(dmap.fg$dist))+
  theme_bw(20)+coord_equal()
```

***

```{r, fig.cap="Cell Distance Map",fig.height=6}
ggplot(thick.map(dmap.fg),aes(x=x,y=y,fill=sph.rad))+
  geom_tile(color="black")+
  labs(title="Single Cell",fill="Sphere\nRadius")+
  th_fillmap.fn(max(dmap.fg$dist))+
  theme_bw(20)+coord_equal()
```

Thickness Map
===
```{r, fig.cap="Cell Distance Map",fig.height=6}
dmap.bg<-dist.map(fill.img,fg.ph=0,bg.ph=155)
ggplot(dmap.bg,aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Single Cell",fill="Distance\nFrom\nSurface")+
  th_fillmap.fn(max(dmap.bg$dist))+
  theme_bw(20)+coord_equal()
```

***

```{r, fig.cap="Cell Distance Map",fig.height=6}
ggplot(thick.map(dmap.bg),aes(x=x,y=y,fill=sph.rad))+
  geom_tile(color="black")+
  labs(title="Single Cell",fill="Sphere\nRadius")+
  th_fillmap.fn(max(dmap.bg$dist))+
  theme_bw(20)+coord_equal()
```

Thickness Map From Skeleton
===

Calculating the distance map by drawing a sphere at each point is very time consuming ($O(n^3)$). 

- The skeleton (last section) is very closely related to the thickness. 
- We found the local maxima in the image using the Laplace
- We can thus grow the Spheres from these points instead of all
- Start by instead of thresholding transforming the image to the distance at each point

$$ \textrm{thSkelImage}(x,y) = $$
$$ \begin{cases}
\textrm{cImg}_1(x,y) , & \textrm{cImg}_1(x,y)\geq MIN-DIST \\ 
  &  \& \textrm{ cImg}_2(x,y)\geq MIN-SLOPE \\
0, & \textrm{otherwise}
\end{cases}$$ 

***

```{r, fig.cap="Skeleton",fig.height=9}

merge.mesh.dif<-rbind(
  cbind(subset(test.mesh.dist,dist>=sqrt(3)),type="Non-surface\nPoints",val=0),
  cbind(subset(test.mesh.dif,val>.5),type="Slope\nPoints",dist=0)
  )
overlap.im<-ddply(merge.mesh.dif,.(x,y),function(c.pts) {
  data.frame(cnt=nrow(c.pts), # points from how many images are present
             dist=max(c.pts$dist))
})

th_fillmap<-scale_fill_gradientn(colours=rainbow(10),limits=c(0,max(test.mesh.dist$dist)))
ggplot(subset(overlap.im,cnt==2),aes(x=x,y=y,fill=dist))+
  geom_tile(color="black",alpha=1)+
  labs(title="Value Skeleton",fill="Source")+
  th_fillmap+
  theme_bw(20)+coord_equal()
```


From Skeleton vs All Points
===

```{r, fig.cap="Thickness from skeleton",fig.height=6,results='markup'}

filled.skel<-fill.img.fn(subset(overlap.im,cnt==2),step.size=0.25,dist=0,cnt=0)
system.time(skthmap<-thick.map(filled.skel))
ggplot(skthmap,aes(x=x,y=y,fill=sph.rad))+
  geom_tile(color="black")+
   th_fillmap+
  labs(title="Thickness From Skeleton",fill="Sphere\nRadius")+
  theme_bw(20)+coord_equal()
```

***

```{r, fig.cap="Thickness from all points",fig.height=6,results='markup'}
system.time(thmap<-thick.map(test.mesh.dist))
ggplot(thmap,aes(x=x,y=y,fill=sph.rad))+
  geom_tile(color="black")+
  labs(title="Full Thickness Map",fill="Sphere\nRadius")+
  th_fillmap+
  theme_bw(20)+coord_equal()
```

Statistically Does it Matter
===

```{r, fig.cap="Thickness from all points",fig.height=6}

hist.sum<-ddply.cutcols(rbind(
  cbind(thmap,type="Full Map"),
  cbind(skthmap,type="Skeleton Map")
  ),
  .(cut_interval(sph.rad,12),type),
  function(c.block) data.frame(count=nrow(c.block))
  )
ggplot(hist.sum,aes(x=sph.rad,color=type,y=count))+
  geom_line()+geom_point()+
  labs(x="Sphere Radius",y="Voxel Count",color="Algorithm")+
  theme_bw(20)+scale_y_sqrt()
```

***

#### It depends

- Small structures are lost
- They might not have been very important or _noisy_ anyways
- Higher values are very similar

```{r, results='asis'}
comp.table<-data.frame(cbind(summary(thmap$sph.rad),summary(skthmap$sph.rad)))
names(comp.table)<-c("Full Map","Skeleton Map")
kable(comp.table)
```


How much can we cut down
===

```{r, fig.cap="Skeleton",fig.height=6,results='markup'}
merge.mesh.dif<-rbind(
  cbind(subset(test.mesh.dist,dist>2),type="Non-surface\nPoints",val=0),
  cbind(subset(test.mesh.dif,val>1.25),type="Slope\nPoints",dist=0)
  )
overlap.im<-ddply(merge.mesh.dif,.(x,y),function(c.pts) {
  data.frame(cnt=nrow(c.pts), # points from how many images are present
             dist=max(c.pts$dist))
})
tfilled.skel<-fill.img.fn(subset(overlap.im,cnt==2),step.size=0.25,dist=0,cnt=0)
system.time(tskthmap<-thick.map(tfilled.skel))
ggplot(tskthmap,aes(x=x,y=y,fill=sph.rad))+
  geom_tile(color="black")+
   th_fillmap+
  labs(title="Thickness from Skeleton\n Slope>1.25 & Dist>1.7",fill="Sphere\nRadius")+
  theme_bw(20)+coord_equal()
```

***

```{r, fig.cap="Thickness Distributions Compared",fig.height=4}
hist.sum<-ddply.cutcols(rbind(
    cbind(thmap,type="Full Map"),
    cbind(skthmap,type="Skeleton Map"),
    cbind(tskthmap,type="Tiny Skeleton Map")),
  .(cut_interval(sph.rad,12),type),
  function(c.block) data.frame(count=nrow(c.block))
  )
ggplot(hist.sum,aes(x=sph.rad,color=type,y=count))+
  geom_line()+geom_point()+
  labs(x="Sphere Radius",y="Voxel Count",color="Algorithm")+
  theme_bw(20)+scale_y_sqrt()
```

```{r, results='asis'}
comp.table<-data.frame(cbind(summary(thmap$sph.rad),summary(skthmap$sph.rad),summary(tskthmap$sph.rad)))
names(comp.table)<-c("Full Map","Skeleton Map","Tiny Skeleton Map")
kable(comp.table)
```

Thickness in 3D Images
===

While the examples and demonstrations so far have been shown in 2D, the same exact technique can be applied to 3D data as well. For example for this liquid foam structure

![Liquid Foam Plateau Border](ext-figures/3dfoam.png)

***
- The thickness can be calculated of the background (air) voxels in the same manner.

- With care, this can be used as a proxy for bubble size distribution in systems where all the bubbles are connected to it is difficult to identify single ones.


![Liquid Foam Thickness](ext-figures/thk3dfoam.png)


Watershed
===

Watershed is a method for segmenting objects without using component labeling. 
- It utilizes the shape of structures to find objects
- From the distance map we can make out substructures with our eyes
- But how to we find them?!

***

```{r, fig.cap="Closer Circles",fig.height=9}
test.sph.list.big<-test.sph.list
# bring everything closer
test.sph.list.big$xi<-0.95*test.sph.list.big$xi
test.sph.list.big$yi<-0.95*test.sph.list.big$yi

ws.distmap<-dist.map(make.sph.img(test.sph.list.big,seq(-10,10)),fg.ph=1,bg.ph=0)

ggplot(ws.distmap,aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  labs(title="Original Distance Map",fill="Distance\nFrom\nSurface")+
  theme_bw(20)+coord_equal()
```

Watershed: Flowing Downhill
===

So we start by distributing pixels all across the image
```{r, fig.cap="Closer Circles",fig.height=9}

ws.distmap$count<-1 # evenly distributed
ws.distmap$orig.x<-ws.distmap$x
ws.distmap$orig.y<-ws.distmap$y
ggplot(ws.distmap,aes(x=x,y=y,fill=dist))+
  geom_tile(color="black")+
  geom_tile(aes(color=as.factor(count)),fill="red",alpha=0.5,size=.2)+
  labs(title="Initial Distribution of Water",fill="Distance\nFrom\nSurface",
       color="Water\nCount")+
  theme_bw(20)+coord_equal()
```

***
If any of the neighbors have a higher distance (more downhill) then move to that position
```{r watershed_code}
single.watershed<-function(ws.dmap,s.dist=1) {
  ws.fun<-function(in.pos) {
    c.pos<-in.pos[1,]
    # all the points within range of the current voxel with a greater distance
    # since everything flows uphill
    jump.pts<-subset(ws.dmap,
           (sqrt( (x-c.pos$x)^2 + (y-c.pos$y)^2 )<=s.dist) & dist>=c.pos$dist)
    jump.pts<-jump.pts[order(-jump.pts$dist),]
    # find the furtherst point and
    # save the old positions
    best.pts<-cbind(jump.pts[1,c("x","y","dist")],
                    orig.x=in.pos$orig.x,
                    orig.y=in.pos$orig.y,
                    count=in.pos$count)

    # leave an empty copy of the voxel behind (c.pos)
    c.pos$count<-0
    rbind(best.pts,
          c.pos)
    
  }
  rbind(
    ddply(subset(ws.dmap,count>0),.(x,y),ws.fun),
    subset(ws.dmap,count==0)
    )
} 

prop.watershed<-function(...,iters=1) {
  if(iters>1) prop.watershed(single.watershed(...),iters=iters-1)
  else single.watershed(...)
}
show.watershed<-function(ju.tab) 
  ddply(ju.tab,.(x,y),function(c.pos) data.frame(dist=c.pos[1,"dist"],count=sum(c.pos[,"count"])))
```
```{r, fig.cap="Closer Circles",fig.height=9}

ws.onestep<-prop.watershed(ws.distmap,sqrt(2),iters=1)
ggplot(show.watershed(ws.onestep),aes(x=x,y=y,fill=dist))+
  geom_tile(aes(fill=as.factor(count),color=dist),size=.2)+
  labs(title="After one step",color="Distance\nFrom\nSurface",
       fill="Water\nCount")+
  theme_bw(20)+coord_equal()
```

Watershed: More flowing
===
```{r, fig.cap="Closer Circles",fig.height=9}

ws.fewstep<-prop.watershed(ws.onestep,sqrt(2),iters=1)
ggplot(show.watershed(ws.fewstep),aes(x=x,y=y,fill=dist))+
  geom_tile(aes(fill=(count),color=dist),size=1)+
  labs(title="After 2 steps",color="Distance\nFrom\nSurface",
       fill="Water\nCount")+
   scale_fill_gradientn(colours=rainbow(10))+
  theme_bw(20)+coord_equal()
```

***

```{r, fig.cap="Closer Circles",fig.height=9}
ws.manysteps<-prop.watershed(ws.fewstep,sqrt(2),iters=4)
ggplot(show.watershed(ws.manysteps),aes(x=x,y=y,fill=dist))+
  geom_tile(aes(fill=as.factor(count),color=dist),size=.3)+
  labs(title="After 6 steps",color="Distance\nFrom\nSurface",
       fill="Water\nCount")+
  theme_bw(20)+coord_equal()
```

Watershed: Regrowing
===
We can then take the points from these basins and regrow them back to their original size and these represent the watershed regions of our image

***

```{r, fig.cap="Closer Circles",fig.height=9}
ws.manysteps$label<-with(ws.manysteps,paste("Cent:",x,",",y,sep=""))
ggplot(subset(ws.manysteps,count>0),aes(x=orig.x,y=orig.y))+
  geom_tile(aes(fill=label),color="black")+
  labs(title="After 6 steps",
       fill="Watershed")+
  theme_bw(20)+coord_equal()
```

Watershed
===

More complicated shapes

![More Complicated Pattern](ext-figures/DistanceMapToWatershed.png)

Curvature
===

Curvature is a metric related to the surface or interface between phases or objects. 
- It is most easily understood in its 1D sense or being the radius of the circle that matchs the local shape of a curve

![Curvature-Wikipedia](http://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Osculating_circle.svg/250px-Osculating_circle.svg.png)

- Mathematically it is defined as 

$$ \kappa = \frac{1}{R} $$

- Thus a low curvature means the value means a very large radius and high curvature means a very small radius


Curvature: Surface Normal
===

In order to meaningfully talk about curvatures of surfaces, we first need to define a consistent frame of reference for examining the surface of an object. We thus define a surface normal vector as a vector oriented orthogonally to the surface away from the interior of the object $\rightarrow \vec{N}$


![Surface](ext-figures/3Dsurface.png)
![Surface Normals](ext-figures/3DsurfaceWiNormals.png)

Curvature: 3D
===
With the notion of surface normal defined ($\vec{N}$), we can define many curvatures at point $\vec{x}$ on the surface. 
- This is because there are infinitely many planes which contain both point $\vec{x}$ and $\vec{N}$ 
- More generally we can define an angle $\theta$ about which a single plane containing both can be freely rotated
- We can then define two principal curvatures by taking the maximum and minimum of this curve

$$ \kappa_1 = \textrm{max}(\kappa(\theta)) $$
$$ \kappa_2 = \textrm{min}(\kappa(\theta)) $$

***

### Mean Curvature
The mean of the two principal curvatures
$$ H = \frac{1}{2}(\kappa_1+\kappa_2) $$

### Gaussian Curvature

The mean of the two principal curvatures
$$ K = \kappa_1\kappa_2 $$
- positive for spheres (or spherical inclusions)
 - curvatures agree in sign
- negative for saddles (hyperboloid surfaces)
 - curvatures disgree in sign
- 0 for planes

Curvature: 3D Examples
===
Examining a complex structure with no meaningful ellipsoidal or watershed model. The images themselves show the type of substructures and shapes which are present in the sample. 


![Mean Curvature](ext-figures/Complicated3DsurfaceMCurv.png)


***

- gaussian curvature:  the comparative amount of surface at, above, and below 0 
 - from spherical particles into annealed mesh of balls

![Gaussian Curvature](ext-figures/Complicated3DsurfaceGCurv.png)



Characteristic Shape
===

Characteristic shape can be calculated by measuring principal curvatures and normalizing them by scaling to the structure size. A distribution of these curvatures then provides shape information on a structure indepedent of the size. 

For example a structure transitioning from a collection of perfectly spherical particles to a annealed solid will go from having many round spherical faces with positive gaussian curvature to many saddles and more complicated structures with 0 or negative curvature.


Curvature: Take Home Message
===
It provides another metric for characterizing complex shapes
- Particularly useful for examining interfaces
 - Folds, saddles, and many other types of points are not characterized well be ellipsoids or thickness maps
- Provides a scale-free metric for assessing structures
- Can provide visual indications of structural changes

Other Techniques
===

There are hundreds of other techniques which can be applied to these complicated structures, but they go beyond the scope of this course. Many of them are model-based which means they work well but only for particular types of samples or images. Of the more general techniques several which are easily testable inside of FIJI are 
- Directional Analysis = Looking at the orientation of different components using Fourier analysis (_Analyze_ $\rightarrow$ _Directionality_)
- Tubeness / Surfaceness (_Plugins_ $\rightarrow$ _Analyze_ $\rightarrow$) characterize binary images and the shape at each point similar to curvature but with a different underlying model

*** 
- Fractal Dimensionality = A metric for assessing the structure as you scale up and down by examining various spatial relationships
 - Ma, D., Stoica, A. D., & Wang, X.-L. (2009). Power-law scaling and fractal nature of medium-range order in metallic glasses. Nature Materials, 8(1), 30–4. doi:10.1038/nmat2340
- Two (or more) point correlation functions = Used in theoretical material science and physics to describe random materials and can be used to characterize distances, orientations, and organization in complex samples 
 - Jiao, Y., Stillinger, F., & Torquato, S. (2007). Modeling heterogeneous materials via two-point correlation functions: Basic principles. Physical Review E, 76(3). doi:10.1103/PhysRevE.76.031110
 - Andrey, P., Kiêu, K., Kress, C., Lehmann, G., Tirichine, L., Liu, Z., … Debey, P. (2010). Statistical analysis of 3D images detects regular spatial distributions of centromeres and chromocenters in animal and plant nuclei. PLoS Computational Biology, 6(7), e1000853. doi:10.1371/journal.pcbi.1000853
 - Haghpanahi, M., & Miramini, S. (2008). Extraction of morphological parameters of tissue engineering scaffolds using two-point correlation function, 463–466. Retrieved from http://portal.acm.org/citation.cfm?id=1713360.1713456
 
 
Texture Analysis
===

http://murphylab.web.cmu.edu/publications/boland/boland_node26.html

Sample Textures
===

```{r}
text_imgs<-expand.grid(x=1:16,y=1:16) %>%
  mutate(val_x=x%%2,
         val_y=y%%2,
         val_xy=(x%%2+y%%2)/2.0,
         val_x2y=(x%%4+y%%2)/2.5,
         val_x2y2=(x%%4+y%%3)/3.5,
         val_xya=(x+y)%%3/2.0) %>%
  melt(id.vars=c("x","y"))
text_imgs %>%  ggplot(aes(x,y))+
  geom_raster(aes(fill=value))+
  facet_wrap(~variable)+
  coord_equal()+
  theme_bw(10)
```

```{r}
text_imgs %>% ggplot(aes(value))+
  geom_density(aes(fill=variable),alpha=0.5)+
  theme_bw(10)
```

Co-occurrence matrix
===

$$
C_{\Delta x, \Delta y}(i,j)=\sum_{x=1}^n\sum_{y=1}^m\begin{cases} 1, & \text{if }I(x, y)=i\text{ and }I(x+\Delta x, y+\Delta y)=j \\ 0, & \text{otherwise}\end{cases}
$$

- As text $\rightarrow$ for every possible offset $\vec{r}$ = ($\Delta x$, $\Delta y$) given the current intensity ($i$) how likely is it that value at a point offset by that amount is $j$.
- This is similar to the two point correlation function but is a 4D representation instead of just a 2D
- In order to use this we need to discretize the image into bins

```{r}
bin_it<-function(x) as.numeric(as.factor(cut_interval(x,4)))
bin_image<-text_imgs %>% 
  mutate(value=bin_it(value)) 
bin_image %>%
  group_by(variable,value) %>%
  summarize(count=n()) %>%
  ggplot(aes(value,count))+
  geom_bar(aes(fill=variable),alpha=0.5,stat="identity",pos="dodge")+
  facet_grid(~variable)+
  theme_bw(10)
```

Calculating the Matrix
===

```{r}
bin_image %>% 
  merge(bin_image,by=c("variable")) %>% 
  mutate(dx=x.y-x.x,dy=y.y-y.x) %>% 
  subset((abs(dx)<4) & (abs(dy)<4)) %>%
  group_by(variable,dx,dy,value.x,value.y) %>%
  summarize(count=n()) %>%
  mutate(ij=paste0(value.x,"->",value.y)) %>%
  ggplot(aes(dx,dy))+
  geom_raster(aes(fill=count))+
  facet_grid(variable~ij)+
  theme_bw(10)
```

Overlapping
===

```{r}
bin_image %>% 
  merge(bin_image,by=c("variable")) %>% 
  mutate(dx=x.y-x.x,dy=y.y-y.x) %>% 
  subset((abs(dx)<5) & (abs(dy)<5)) %>%
  group_by(variable,dx,dy,value.x,value.y) %>%
  summarize(count=n()) %>%
  mutate(ij=paste0(value.x,"->",value.y)) %>%
  ggplot(aes(dx,dy))+
  geom_contour(aes(z=count,color=ij))+
  facet_wrap(~variable)+
  theme_bw(10)
```

Simple Correlation
===
Using the mean difference ($E[i-j]$) instead of all of the i,j pair possiblities 

```{r}
bin_image %>% 
  merge(bin_image,by=c("variable")) %>% 
  mutate(dx=x.y-x.x,dy=y.y-y.x,dv=value.y-value.x) %>% 
  subset((abs(dx)<4) & (abs(dy)<4)) %>%
  group_by(variable,dx,dy,value.x,value.y) %>%
  summarize(score=mean(dv)) %>%
  ggplot(aes(dx,dy))+
  geom_raster(aes(fill=score))+
  facet_wrap(~variable)+
  coord_equal()+
  theme_bw(10)
```

Radial Coordinates
===
Rather than using $\Delta x$ and $\Delta y$ we can also show the relative positions 

```{r}
bin_image %>% 
  merge(bin_image,by=c("variable")) %>% 
  mutate(dr=sqrt((x.y-x.x)**2+(y.y-y.x)**2),
         dtheta=180/pi*atan((y.y-y.x)/(x.y-x.x)),
         dv=value.y-value.x) %>% 
  subset(dr<6 & !is.na(dtheta)) %>%
  mutate(dr=cut_interval(dr,10),
         dtheta=cut_interval(dtheta,8)) %>%
  group_by(variable,dr,dtheta,value.x,value.y) %>%
  summarize(score=mean(dv)) %>%
  ggplot(aes(dtheta,dr))+
  geom_tile(aes(fill=score))+
  facet_wrap(~variable)+
  coord_polar(theta="x")+
  scale_fill_distiller(palette = "Spectral")+
  theme_bw(10)
```